In [1]:
import lxml.html
import time
import re
from tqdm import tqdm
from stanfordnlp.server import CoreNLPClient
import pickle
from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

from pathlib import Path



In [2]:
p_path = './data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/'
p = Path(p_path)
x= list(p.glob('*.html'))
document_IDs = [x_.name[:-5] for x_ in x]

doc = [[]]*len(document_IDs)
for i, document_ID in tqdm(enumerate(document_IDs)):
    document_path = p_path + document_ID + '.html'
    doc_ = find_identifier_definition(document_path)
    doc[i] = doc_
#     start_time = time.time()
#     doc_ = XMLDocument(document_path)
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))

#     doc_.processor()
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))

#     doc_.extract_identifiers()
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))

#     doc_.extract_sentences()
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))

#     doc_.POS_tagging()
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))

#     doc_.pattern_based_extract_description()
#     print("elapsed time : {0:.7f} seconds ---".format(time.time()-start_time))
    
#     doc[i] = doc_

5it [01:52, 22.51s/it]


elapsed time : 0.0001562 seconds ---
process document
elapsed time : 0.0775740 seconds ---
Number of math components is 278
elapsed time : 1.5491769 seconds ---
elapsed time : 4.1357479 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-117166a372f84172.props -preload tokenize,ssplit,pos
elapsed time : 9.3793600 seconds ---
elapsed time : 9.4315441 seconds ---
elapsed time : 0.0000069 seconds ---
process document
elapsed time : 0.0246732 seconds ---
Number of math components is 68
elapsed time : 0.1343839 seconds ---
elapsed time : 46.1531560 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 

In [3]:
# with open('doc_1806.09460.binaryfile', 'rb') as f:
#     doc = pickle.load(f)
with open('./output/docs.binaryfile', 'wb') as f:
    pickle.dump(doc, f)

In [4]:
document_IDs

['1502.00338', '1903.01808', '1806.08999', '0808.0311', '1806.09460']

In [5]:
[doc[i].path for i in range(5)]

['./data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/1502.00338.html',
 './data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/1903.01808.html',
 './data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/1806.08999.html',
 './data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/0808.0311.html',
 './data/resources/dataset-arXMLiv-08-2019/process-control_physical-model/1806.09460.html']

In [6]:
with open('identifiers_1.txt', 'w') as f:
    f.write('\n'.join(doc[1].identifiers))

In [7]:
len(doc[4].sentences)

111

In [8]:
len(doc[4].identifiers)

111

In [9]:
len(doc[4].tagged_sentence_list)

111

In [10]:
sentences = doc[4].sentences
tagged_sentence_list = doc[4].tagged_sentence_list

In [11]:
sentences[1][1]
import numpy as np

In [12]:
np.shape(sentences)

(111,)

In [13]:
A = [(1,2), (3,4)]

In [14]:
B = []

In [15]:
B.append(A)

In [16]:
B

[[(1, 2), (3, 4)]]

In [17]:
np.shape(tagged_sentence_list[0])

(408,)

In [18]:
for math_id, (sentence_i, tagged_sentence_list_i) in zip(sentences, tagged_sentence_list):
    identifier = 'MATH{:04d}'.format(math_id_)
    
    if not tagged_sentence_list_i:
        continue
        
    description_candidate_ = []
    for tagged_sentence_list_i_ in tagged_sentence_list_i:
        indexes_target = [n for n, v in enumerate(tagged_sentence_list_i_) if v == (identifier, 'NN')]
        for index_target in indexes_target:
            # 1. <description> <identifier>
            description=[]
            for i in range(index_target-1, -1, -1):
                (description_, pos_) = tagged_sentence_list_i_[i]
                if (not 'MATH' in description_) and reg_description.fullmatch(pos_):
                    description.append(description_)
                else:
                    break
            if description:
                description_candidate_.append((' '.join(description), ' '.join(description) + ' ' + identifier))

            # 2. <identifier> is <description>
            # 3. <identifier> is the <description>
            description=[]
            if tagged_sentence_list_i_[index_target+1][0]=='is':
                mid_pattern = ' is '
                if tagged_sentence_list_i_[index_target+2][0]=='the':
                    index_start = index_target+3
                    mid_pattern+='the '
                else:
                    index_start = index_target+2
                for i in range(index_start, len(tagged_sentence_list_i_)):
                    (description_, pos_) = tagged_sentence_list_i_[i]
                    if (not 'MATH' in description_) and reg_description.fullmatch(pos_):
                        description.append(description_)
                    else:
                        break
                if description:
                    description_candidate_.append((' '.join(description), identifier + mid_pattern + ' '.join(description)))

            # 4. let <identifier> be the <description>
            if (tagged_sentence_list_i_[index_target-1][0]=='let') and (tagged_sentence_list_i_[index_target+1][0]=='be') and (tagged_sentence_list_i_[index_target+2][0]=='the'):
                for i in range(index_target+3, len(tagged_sentence_list_i_)):
                    (description_, pos_) = tagged_sentence_list_i_[i]
                    if (not 'MATH' in description_) and reg_description.fullmatch(pos_):
                        description.append(description_)
                    else:
                        break
                if description:
                    description_candidate_.append((' '.join(description), 'let ' + identifier + ' be the ' + ' '.join(description)))

            # 5. <description> is|are denoted by <identifier>
            if (tagged_sentence_list_i_[index_target-1][0]=='by') and (tagged_sentence_list_i_[index_target-2][0]=='denoted') and (tagged_sentence_list_i_[index_target-3][0]==('is' or 'are')):
                for i in range(index_target-4, -1, -1):
                    (description_, pos_) = tagged_sentence_list_i_[i]
                    if (not 'MATH' in description_) and reg_description.fullmatch(pos_):
                        description.append(description_)
                    else:
                        break
                if description:
                    description_candidate_.append((' '.join(description), ' '.join(description) + ' is|are denoted by ' + identifier))
                    description=[]

            # 6. <identifier> denotes */DT <description>
            if (tagged_sentence_list_i_[index_target+1][0]=='denotes') and (tagged_sentence_list_i_[index_target+2][1]=='DT'):
                for i in range(index_target+3, len(tagged_sentence_list_i_)):
                    (description_, pos_) = tagged_sentence_list_i_[i]
                    if (not 'MATH' in description_) and reg_description.fullmatch(pos_):
                        description.append(description_)
                    else:
                        break
                if description:
                    description_candidate_.append((' '.join(description), identifier + ' denotes */DT ' + ' '.join(description)))


ValueError: too many values to unpack (expected 2)

In [ ]:

X = [[i, j] for i in range(10) for j in range(10,20)]

In [ ]:
for i, (x,y,z) in enumerate(zip([1,2,3],[2,3,4],[3,4,5])):
    print(i)
    print(x, y, z)

In [ ]:
sum([len(sentence.split()) for sentences_ in sentences for sentence in sentences_ ])

In [ ]:
sum([len(v) for v in tagged_sentence_list])

In [ ]:
tagged_sentence_list[0][0]

In [ ]:
sentences[0][0].split()